In [1]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import json
from datasets import Dataset
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments, Trainer

In [2]:
# Load TinyLlama
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set pad token

# Quantize model to 4-bit (saves VRAM)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [3]:
from datasets import Dataset
import json

# 1. Load your custom dataset
with open("data/custom_dataset_advanced.jsonl", "r") as f:
    data = [json.loads(line) for line in f]

# Convert to Hugging Face Dataset
dataset = Dataset.from_list(data)

def tokenize_function(examples):
    texts = [
        f"### Context:\n{ctx}\n\n### Instruction:\n{inst}\n\n### Response:\n{out}"
        for ctx, inst, out in zip(examples["context"], examples["instruction"], examples["output"])
    ]
    tokenized = tokenizer(texts, truncation=True, max_length=512, padding="max_length")
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    batch_size=4,
    remove_columns=["instruction", "output"]  # Remove original columns
)

# 4. Verify
print(tokenized_dataset[0].keys())  # Should show: ['input_ids', 'attention_mask']



Map:   0%|          | 0/13 [00:00<?, ? examples/s]

dict_keys(['context', 'input_ids', 'attention_mask', 'labels'])


In [4]:
# LoRA settings
lora_config = LoraConfig(
    r=8,                  # Rank
    lora_alpha=32,        # Scaling factor
    target_modules=["q_proj", "v_proj"],  # TinyLlama layers to target
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Should show ~0.1% trainable params



trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [5]:
def tokenize_function(examples):
    # Combine context, instruction, and output into the prompt
    texts = [
        f"### Context:\n{ctx}\n\n### Instruction:\n{inst}\n\n### Response:\n{out}"
        for ctx, inst, out in zip(examples["context"], examples["instruction"], examples["output"])
    ]
    
    tokenized = tokenizer(
        texts,
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors="np"  # Returns NumPy arrays (compatible with Trainer)
    )
    
    # For causal LM, labels = input_ids (predict next token)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# Apply to dataset
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    batch_size=4,
    remove_columns=["instruction", "output", "context"]  # Remove original columns
)

# Verify
print(tokenized_dataset[0].keys())  # Should show: ['input_ids', 'attention_mask', 'labels']

Map:   0%|          | 0/13 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [6]:
training_args = TrainingArguments(
    output_dir="./tinyllama-lora-trailtracker",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=2e-5,  # Slightly lower for stability with context
    num_train_epochs=3,
    logging_steps=10,
    save_steps=50,
    fp16=True,
    optim="paged_adamw_8bit",
    remove_unused_columns=False,  # Required when using custom tokenization
    report_to="none",
)

# Simplified Trainer (no custom data collator needed)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,  # Uses pre-tokenized data with labels
)

# Start training
trainer.train()

# Save adapter
model.save_pretrained("./tinyllama-lora-trailtracker-final")

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss


In [7]:
# Save the LoRA adapter (no need to save base model)
model.save_pretrained("./outputs/tinyllama-lora-trailtracker")
tokenizer.save_pretrained("./outputs/tinyllama-lora-trailtracker")  # Optional but recommended

('./outputs/tinyllama-lora-trailtracker\\tokenizer_config.json',
 './outputs/tinyllama-lora-trailtracker\\special_tokens_map.json',
 './outputs/tinyllama-lora-trailtracker\\tokenizer.model',
 './outputs/tinyllama-lora-trailtracker\\added_tokens.json',
 './outputs/tinyllama-lora-trailtracker\\tokenizer.json')

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch

# 1. Clear VRAM
torch.cuda.empty_cache()

# 2. Quantization config (same as training)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

# 3. Load components
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
tokenizer.pad_token = tokenizer.eos_token

# 4. Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    quantization_config=bnb_config,
    device_map="auto",
    low_cpu_mem_usage=True
)

# 5. Load LoRA adapter (now with context support)
model = PeftModel.from_pretrained(
    base_model,
    "./outputs/tinyllama-lora-trailtracker",  # Updated path
    device_map="auto"
)

# Optional: Merge for faster inference (uses more VRAM)
# model = model.merge_and_unload()
# model.save_pretrained("./outputs/tinyllama-merged")  # If merging

In [9]:
import re

def generate_response(instruction, context=""):
    # Format the prompt with context
    prompt = f"### Context:\n{context}\n\n### Instruction:\n{instruction}\n\n### Response:\n"
    
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to("cuda")
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )
    
    # Extract and clean the response
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response_text = full_response.split("### Response:")[-1].strip()
    response_text = re.sub(r'(?<!\d)\.(?!\d)', '.\n', response_text)  # Add newlines after periods
    
    return response_text

# Example tests
print("Test 1 (with context):")
print(generate_response(
    "Who created TrailTracker?",
    "Developed at Nackademin by Robert and Olle"
))

print("\nTest 2 (without context):")
print(generate_response("Describe TrailTracker's pricing"))

Test 1 (with context):
TrailTracker is a free and open-source application that helps people find hiking trails, routes, and trails in their local area.
 It is available for both Android and

Test 2 (without context):
TrailTracker offers a free trail tracker app for users who want to keep track of their hiking and trail running activities.
 The app is free to download and use, and users can create an account to access the rest of the features.
 Users can create custom tracks, set up notifications for events like trail races or fitness challenges, and share their tracks with friends and family.


The app offers a basic free plan with limited features, such as the ability to create and share custom tracks, but users will need to pay for the premium plan for more advanced features like advanced analytics and personalized recommendations.
 The premium plan costs $10 per month or $100 per year, depending on the user's plan.


Overall, TrailTracker's pricing is competitive and reasonable comp

In [10]:
print("\nTest 3 (without context):")
print(generate_response("Tell me about TrailTracker"))


Test 3 (without context):
TrailTracker is a mobile app that tracks your hiking or running adventures.
 The app allows you to log your activities and view your progress over time.
 You can also connect with other hikers and runners to share your experiences and get recommendations for new trails.
 The app is available on both Android and iOS devices and is free to download.
 It also features a social media component that allows you to share your hiking or running experiences with friends and followers.
 Overall, TrailTracker is a great tool for anyone who loves hiking or running and wants to track their progress over time.



In [11]:
print("Test 4 (with context):")
print(generate_response(
    "Tell me about TrailTracker",
    "Webapp for hike planning"
))

Test 4 (with context):
TrailTracker is a web app that allows users to create and manage their hiking trips.
 The app provides a user-friendly interface for creating and managing hiking routes, including the ability to add points of interest (POIs) such as trails, mountains, and waterfalls.
 Users can also customize their profiles, such as adding their hiking experience level and preferred hiking routes.
 The app also includes a map view that shows the user's current location and nearby POIs.
 Overall, TrailTracker aims to provide a seamless hiking experience by making it easy for users to plan, track, and share their hikes.



In [13]:
print("Test 5 (with context):")
print(generate_response(
    "What is the main feature of TrailTracker?",
    "Webapp for hike planning"
))

Test 5 (with context):
TrailTracker is a web application that allows users to plan and track their hikes.
 Users can create profiles, add hikes, and set goals for their hikes.
 They can also connect with other hikers and share their hikes with them.
 The main feature of TrailTracker is the ability to plan and track hikes.
 Users can create profiles, add hikes, and set goals for their hikes.
 They can also connect with other hikers and share their hikes with them.
 Users can also view the progress of their hikes and see how many miles they have covered.

